In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient
from datetime import datetime
import csv
from google_trans_new import google_translator
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0 
from collections import Counter
from bson.objectid import ObjectId
import unicodedata
import urllib.parse
from difflib import SequenceMatcher

In [32]:
client = MongoClient('mongodb://192.168.1.11:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']
coleccion_app_novela = db['app_novela']
coleccion_app_capitulo = db['app_capitulo']
sitio_id = '65e7e1707963e189aa4c1256'

In [3]:
def traducir(texto: str) -> str:
    """Traduce texto usando múltiples servicios de traducción"""
    translators = [
        ('bing', lambda t: ts.translate_text(t, translator='bing', to_language='es')),
        ('google', lambda t: ts.translate_text(t, translator='google', to_language='es')),
        ('google_new', lambda t: google_translator().translate(t, lang_tgt='es'))
    ]
    
    for name, func in translators:
        try:
            return func(texto)
        except Exception as e:
            print(f"Fallo en {name}: {e}")
            continue
    return texto

In [27]:
try:
    df_list_novels = pd.read_csv(r'D:/PycharmProjects/novelasligeraswebscrapping/daily_novel_bin_novelbin.csv', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    # df_list_novels['fecha']=datetime.now()
except:
    df_list_novels = pd.DataFrame(columns=['nombre', 'url', 'procesada', 'fecha'])
# df_list_novels
# df_list_novels = pd.DataFrame(columns=['nombre', 'url', 'procesada'])
# df_existing_novels =pd.read_csv('existing_novels.csv', encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
# df_existing_novels

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [6]:
url = 'https://novel-bin.net/sort/daily-novel-bin'
# url='https://novel-bin.net/sort/hot-novel-bin'
# url = 'https://novel-bin.net/sort/completed-novel-bin'
# url='https://novel-bin.net/sort/popular-novel-bin'

In [7]:
driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [8]:
driver.get(url)
# driver.maximize_window()

In [ ]:
(int(driver.find_element(By.XPATH,'/html/body/div/main/div[3]/div/ul/li[9]/a').get_attribute('href').split('=')[-1])+1)

In [10]:
if df_list_novels.empty:
#     time.sleep(3)
    list_novels = {"nombre":[],"url":[]}
    page = 2
    limit = (int(driver.find_element(By.XPATH,'/html/body/div/main/div[3]/div/ul/li[9]/a').get_attribute('href').split('=')[-1])+1)
    while page < limit:
        html = driver.page_source
        soup = bs(html, 'html.parser')
        list_novels_page = soup.find_all('h3', class_='novel-title')
        for a_list in list_novels_page:
            list_novels['nombre'].append(a_list.find('a').getText())
            list_novels['url'].append(a_list.find('a').get('href'))
#             print(a_list.find('a').getText(),a_list.find('a').get('href'))
#         time.sleep(1)
        driver.get(f"{url}?page={page}")
        page+= 1
        print(f'{page}, ', end='') if page%50==0 else print('', end='')
#         break
    df_list_novels = pd.DataFrame(list_novels)
    df_list_novels.to_csv(r'D:/PycharmProjects/novelasligeraswebscrapping/daily_novel_bin_novelbin.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    df_list_novels['procesada']=0
    df_list_novels['fecha']=datetime.now()

In [28]:
df_list_novels

,nombre,url,procesada,fecha
0,A Knight Who Eternally Regresses,https://novel-bin.net/novel-bin/a-knight-who-e...,1.0,2025-03-21 19:46:35.743086
1,Fate in Bleach: Unlimited Blade Works [Isekai],https://novel-bin.net/novel-bin/fate-in-bleach...,1.0,2025-03-21 19:46:35.743086
2,Childhood Friend of the Zenith,https://novel-bin.net/novel-bin/childhood-frie...,1.0,2025-03-21 19:46:35.743086
3,Sword of Coming,https://novel-bin.net/novel-bin/sword-of-coming,1.0,2025-03-21 19:46:35.743086
4,The Great Mage Returns After 4000 Years,https://novel-bin.net/novel-bin/the-great-mage...,1.0,2025-03-21 19:46:35.743086
...,...,...,...,...
8815,A Loner With The Trash Skill 'Abnormal Status ...,https://novel-bin.net/novel-bin/a-loner-with-t...,0.0,2025-03-21 19:46:35.743086
8816,Godly Hunter,https://novel-bin.net/novel-bin/godly-hunter,0.0,2025-03-21 19:46:35.743086
8817,Remember Self-Control: Bossy Husband Loves Me ...,https://novel-bin.net/novel-bin/remember-self-...,0.0,2025-03-21 19:46:35.743086
8818,She Became a Heartbreaker After Entering the N...,https://novel-bin.net/novel-bin/she-became-a-h...,0.0,2025-03-21 19:46:35.743086


In [20]:
def get_novel_data(driver, novel_url):
    global sitio_id
    driver.get(novel_url)
    time.sleep(1)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')

    title = soup_novel.find('h3', class_='title').getText()
    table_info = soup_novel.find('ul', class_='info info-meta').find_all('li')
    author = ''
    genre = ''
    status = ''
    for li in table_info:
        if 'Author:' in li.find('h3').getText().strip().rstrip():
            try:
                author = li.find('a').getText().strip().rstrip()
            except:
                author = ''
        if 'Genre:' in li.find('h3').getText().strip().rstrip():
            try:
                genre = ', '.join([x.getText().strip().rstrip() for x in li.find_all('a')])
            except: 
                genre=''
        if 'Status:' in li.find('h3').getText().strip().rstrip():
            try:
                status = li.find('a').getText().strip().rstrip()
            except:
                status=''

    img_src = soup_novel.find('div', class_='book').find('img').get('src')
    description = soup_novel.find('div', class_='desc-text').getText().strip().rstrip()

    novel_data = {
        "sitio_id": sitio_id,
        "nombre": title.upper(),
        "sinopsis": description,
        "autor": author,
        "genero": genre,
        "status": 'emision' if status == 'Ongoing' else 'completo',
        "url": novel_url,
        "imagen_url": img_src,
        "created_at": datetime.now(),
        "updated_at": datetime.now()
    }

    return novel_data

In [14]:
def get_chapter_data(driver, url_novel, novel_id, collection):
    driver.get(url_novel)
    driver.find_element(By.XPATH, '/html/body/div/main/div[2]/div[1]/div[4]/ul/li[2]/a').click()
    time.sleep(1)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')
    panel_chapter = soup_novel.find('div', class_='panel-body').find_all('div', class_='row')

#     existing_chapters = {x['url'] for x in collection.find({'novela_id': novel_id}, {'url': 1})}
    existing_chapters = {str(x['nombre']).strip().rstrip() for x in collection.find({'novela_id': novel_id}, {'nombre': 1})}

    for row in panel_chapter:
        lists_chapters = row.find_all('ul', class_='list-chapter')
        for ul in lists_chapters:
            a_list = ul.find_all('a')
            for a in a_list:
                chapter_url = str(a.get('href'))
                if str(a.getText()).strip().rstrip() not in existing_chapters:
                    chapter_data = {
                        "novela_id": novel_id,
                        "nombre": str(a.getText()).strip().rstrip(),
                        "url": chapter_url,
                        "created_at": datetime.now(),
                        "updated_at": datetime.now()
                    }
                    collection.insert_one(chapter_data)
#                 else:
#                     print(f"{chapter_url} si estaba en la base de datos.")

In [51]:
# existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({}, {'_id': 1, 'nombre':1, 'url': 1})]
# print(existing_novels[:5])

In [34]:
existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({'sitio_id':sitio_id}, {'_id': 1, 'nombre':1, 'url': 1})]
existing_novels_names = {x['nombre']:x['_id'] for x in existing_novels}
# print(existing_novels_names.values() if "MARTIAL KING'S RETIRED LIFE" in existing_novels_names.keys() else existing_novels_names)
for idx, novel in df_list_novels.iterrows():
    if novel['procesada']==0:
        if str(novel['nombre']).upper().strip().rstrip() in existing_novels_names.keys():
            novel_id = existing_novels_names[str(novel['nombre']).upper().strip().rstrip()]
            print(f"{idx} {novel_id} {str(novel['nombre']).upper().strip().rstrip()} {novel['url']} ya existe, hay que actualizar la URL")
            resultado = coleccion_app_novela.update_one(
                {'_id': ObjectId(novel_id)},
                {'$set': {'url': novel['url']}}
            )
            # print(f'Documentos actualizados: {resultado.modified_count}')
            try:
                get_chapter_data(driver, novel['url'], novel_id, coleccion_app_capitulo)
                # print(f"Todos los capítulos publicados de '{str(novel['nombre']).upper().strip().rstrip()}' han sido ingresados a la Base de Datos.")
            except Exception as e:
                print(f"Error al procesar la novela '{novel['url']}': {e}")
        else:
            print(f"{idx} {str(novel['nombre']).upper().strip().rstrip()} {novel['url']} no existe, hay que crear la novela")
            try:
                novel_data = get_novel_data(driver, novel['url'])
            except:
                driver.get(novel['url'])
                time.sleep(1)
                novel_data = get_novel_data(driver, novel['url'])
            novel_id = str(coleccion_app_novela.insert_one(novel_data).inserted_id)
            print(f"Novela '{novel_data['nombre']}' ingresada a la Base de Datos.")
            try:
                get_chapter_data(driver, novel['url'], novel_id, coleccion_app_capitulo)
                # print(f"Todos los capítulos publicados de '{str(novel['nombre']).upper().strip().rstrip()}' han sido ingresados a la Base de Datos.")
            except Exception as e:
                print(f"Error al procesar la novela '{novel['url']}': {e}")
        df_list_novels.loc[idx, 'procesada']=1
        df_list_novels.to_csv(r'D:/PycharmProjects/novelasligeraswebscrapping/daily_novel_bin_novelbin.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
#     break

#     parsed_url1 = urllib.parse.urlparse(novel['url'])
#     path1 = parsed_url1.path.split('/')[-1]

#     found_match = False
#     for novela in existing_novels:
#         parsed_url2 = urllib.parse.urlparse(novela['url'])
#         path2 = parsed_url2.path.split('/')[-1]

#         similarity = SequenceMatcher(None, path1, path2).ratio()

#         if similarity >= 0.52:
#     if not found_match:

1680 SWORDED AFFAIR https://novel-bin.net/novel-bin/sworded-affair no existe, hay que crear la novela
Novela 'SWORDED AFFAIR' ingresada a la Base de Datos.
1681 SWISS ARMS https://novel-bin.net/novel-bin/swiss-arms no existe, hay que crear la novela
Novela 'SWISS ARMS' ingresada a la Base de Datos.
1682 DISCORDANT NOTE | THE BEGINNING AFTER THE END SI https://novel-bin.net/novel-bin/discordant-note-the-beginning-after-the-end-si no existe, hay que crear la novela
Novela 'DISCORDANT NOTE | THE BEGINNING AFTER THE END SI' ingresada a la Base de Datos.
1683 HEAVEN AND HELLFIRE COMPILED https://novel-bin.net/novel-bin/heaven-and-hellfire-compiled no existe, hay que crear la novela
Novela 'HEAVEN AND HELLFIRE COMPILED' ingresada a la Base de Datos.
1684 65f8c4f773460769e52e4c44 THE HITTING ZONE https://novel-bin.net/novel-bin/the-hitting-zone ya existe, hay que actualizar la URL
1685 CONQUEST OF THE FALLEN: DARK DOMINIONS https://novel-bin.net/novel-bin/conquest-of-the-fallen-dark-dominions

In [35]:
df_existing_novels = pd.DataFrame(existing_novels)
df_existing_novels['fecha'] = datetime.now()

In [36]:
df_existing_novels.to_csv(r'D:/PycharmProjects/novelasligeraswebscrapping/existing_novels.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

In [37]:
driver.quit()